<a href="https://colab.research.google.com/github/bobzinin/python-internals-project/blob/numba/first_sight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numba
numba - библиотека, предназначенная для ускорения высоконагруженных участков кода.

Большую ее функциональность реализуют различные обёртки.
Наиболее популярная из них -- @jit

## @jit args
nopython - будет стараться не прибегать к использованию python кода, а как можно больше генерировать собственный машинный код

cache - закэширует машинный код функции

nogil - для многопоточных приложений

parallel - попытается автоматически распараллелить циклы внутри функции

inline - для использования функции как inline





Одна из вещей, которая даёт значительный прирост производительности --- статическая типизация значений, принимаемых функцией. Вот как можно помочь numba с определением типов:
```python
@decorator([return_type(input_type_1, input_type_2…)])
```


Типы:
- void
- int
- float32/64
- complex64/128
- type[:] -- массивы

Если мы сами не задаём типы, то функция переходит в автоматический режим и компиллируется по мере вызова с ещё неизвестными для неё типами данных(Это замечание верно для большинства декоаторов в библиотеке)

In [144]:
from numba import float64, int32

@jit([float64(float64, float64)]) # функция принимающая 2 float значения и возвращающая другой float
def add(x, y):
    return x + y


@jit([int32(int32, int32), float64(float64, float64)]) # если хотим добавить несколько вариантов
def multiply(x, y):
    return x * y

@jit([float64(float64[:])])
def sum(a):
    s = 0
    for elem in a:
        s += elem
    return s


## @vectorize
векторизация -- процесс, позволяющий за один "такт" обрабатывать по несколько элементов

в нашем случае numba создаёт numpy.ufunc(universal function), которая позволяет быстрее выполнять одинаковые
операции над элементами массива

ufunc быстрее и удобнее за счёт того, что:
- В numpy есть определенные договорённости по работе с многомерными массивами, которые позволяют использовать оптимизированные функции с другими совместимыми
- numba всё равно частично компиллирует нашу функцию в машинный код, поэтому он выполняется быстрее

In [134]:
from numba import vectorize

@vectorize([float64(float64, float64)])
def add(x, y):
    return x + y

## @guvectorize
подходит для функции, зависящей от массива целиком(его части), например: среднее/медиана/стандартное отклонение.

принимает не только типы данных, но и желаемое устройство функции в формате строки:
- (i)->() : вход — вектор длины i, выход — скаляр.
- (i),(i)->() : два вектора одинаковой длины i → скаляр (например, dot).
- (m,n),(n,p)->(m,p) : матричное умножение.

так же функции в таком декораторе не возвращают значений через return, для этого используются массивы, передающиеся в функцию и в которые, по мере выполнения кода, мы записываем результат. Для того, чтобы пометить такие "буфферы"(чтобы компилляторе не применял к ним агрессивные оптимизации), используем:
- writable_args = (0, 1,…) -- кортеж с индексами изменяемых полей в объявлении функции

In [135]:
from numba import guvectorize, int64

@guvectorize([(int64[:], int64, int64[:])], '(n),()->()')
def g(x, y, res):
    acc = 0
    for i in range(x.shape[0]):
        acc += x[i] + y
    res[0] = acc
    
x = np.arange(10)
z = np.array([0])

g(x, 5, z)
print(x)
print(z)

[0 1 2 3 4 5 6 7 8 9]
[95]


## @stencil
позволяет удобно работать с функциями, зависящими от окружения элемента(фильтры на фотографиях/сглаживания и т.п.) - это улучшает читаемость кода

ну и как же без оптимизации с помощью машинного кода

аргументы:
- neighborhood - можно задать то, с какими элементами будет работать наша функция(чтобы вручную задать границы) -- задаём как кортеж кортежей, где внутренние элементы отвечают за минимальный, максимальный отступ по индексу вдоль каждой оси, а внешний -- количество осей
- func_or_mode - показывает, как мы будем обрабатывать границы массива, пока только один режим "constant"
- cval - значение, которое мы присваиваем границам
- standard indexing - используется для выделения массива весов

! порядок указания аргументов в декорируемой функции
- сначала идёт массив, над которым будет происходить работа
- далее массивы(размерами не меньше, чем тот, который мы передали сначала) и скалярные величины

In [136]:
from numba import stencil

@stencil(neighborhood = ((-1, 1),))
def smooth_1d(a):
    return (a[-1] + a[0] + a[1]) / 3.0

@stencil(neighborhood = ((-1, 1), (-1, 1),), standard_indexing=("weights",))
def smooth_2d(a, weights):
    res = 0.
    for i in range(-1, 2):
        for j in range(-1, 2):
            res += weights[i, j] * a[i, j]
    return res
            
img = np.random.rand(5, 5)

weights = np.array([[0.3, 0.5, 0.3], [0.5, 1.0, 0.5], [0.3, 0.5, 0.3]])

print(img)
print("|")
print("\/")
print(smooth_2d(img, weights))

[[0.27585609 0.02901243 0.26111351 0.14409301 0.64690279]
 [0.06416192 0.66836335 0.79237385 0.46835793 0.43318008]
 [0.21992465 0.45230022 0.15971294 0.15794249 0.40603407]
 [0.42285451 0.10725735 0.79110895 0.21593936 0.2723093 ]
 [0.74113012 0.46114102 0.77943439 0.43870995 0.75335656]]
|
\/
[[0.         0.         0.         0.         0.        ]
 [0.         1.33378719 1.29543348 1.60468472 0.        ]
 [0.         1.95363331 1.60009783 1.66895669 0.        ]
 [0.         2.21668192 1.69905242 2.09901154 0.        ]
 [0.         0.         0.         0.         0.        ]]


<>:21: SyntaxWarning: invalid escape sequence '\/'
<>:21: SyntaxWarning: invalid escape sequence '\/'
/var/folders/h2/xk31zvlx1js3dvn28k794qz00000gn/T/ipykernel_7557/2171438089.py:21: SyntaxWarning: invalid escape sequence '\/'
  print("\/")


# Ступени компиляции numba

Используя ключи дебага, можно отследить то, как numba оптимизирует python код

Всего выделяют 8 ключевых ступеней:

1. **Analyze bytecode** - numba получает bytecode, создает CFG (control flow graph) и Data flow graph
2. **Generate IR** - Переводит байт-код в более удобное внутреннее представление numba (intermediate representation)
3. **Rewrite IR** - Переписывает IR с учетом некоторых оптимизаций
4. **Infer types** - Статически типизирует переменные где возможно. Если тип не определяется, вызывается pyobject и происходит откат в object mode
5. **Rewrite typed IR** - Переписывает типизированное промежуточное представление
6. **Perform automatic parallelization** - Автоматическая параллелизация циклов (parfor)
7. **Generate LLVM IR** - Создание LLVM промежуточного представления
8. **Generate machine code** - Генерация нативного машинного кода

**Путь**: Python код → Bytecode → Numba IR → Typed IR → Optimized IR → LLVM IR → Machine Code

In [137]:
import os
import numpy as np
import dis
os.environ['NUMBA_DUMP_CFG'] = '1'
os.environ['NUMBA_DUMP_IR'] = '1'
from numba import jit


In [ ]:
import numpy as np
from numba import int32, float32
from numba.experimental import jitclass

spec = [
    ('value', int32),            
    ('array', float32[:]),       
]

@jitclass(spec)
class Bag(object):
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)

    @property
    def size(self):
        return self.array.size

    def increment(self, val):
        for i in range(self.size):
            self.array[i] += val
        return self.array

    @staticmethod
    def add(x, y):
        return x + y

n = 21
mybag = Bag(n)

In [140]:
@jit
def LightFunction(x, y):
    if y == 0:
        return 0
    return x / y

print(dis.dis(LightFunction))

  1           RESUME                   0

  3           LOAD_FAST                1 (y)
              LOAD_CONST               1 (0)
              COMPARE_OP              88 (bool(==))
              POP_JUMP_IF_FALSE        1 (to L1)

  4           RETURN_CONST             1 (0)

  5   L1:     LOAD_FAST_LOAD_FAST      1 (x, y)
              BINARY_OP               11 (/)
              RETURN_VALUE
None


Что такое CFG(control flow graph) - буквально граф потока управления. То есть программу разбивают на логические блоки и представляют в виде графа, который описывает процесс её исполнения, где показывается из какого блока что может следовать.

Каждый блок кода можно разделить на:
- точку входа
- базовые блоки(тело)
- точку выхода

с помощью точек входа и выхода, блоки связаны между собой.

Что можем увидеть:
- adjacency list - лист зависимостей(какие блоки следуют из данного)
- dominators - блок A доминирует блок B, если любой путь от входа к B проходит через A
- post-dominators - блок A постдоминирует блок B, если любой путь от B к выходу проходит через A
- back edges - обратные рёбра
- loops - циклы
- backbone - блоки, через которые обязательно проходит выполнение

In [141]:
LightFunction(1, 0)

CFG adjacency lists:
{0: [16, 18], 16: [], 18: []}
CFG dominators:
{0: {0}, 16: {16, 0}, 18: {0, 18}}
CFG post-dominators:
{0: {0}, 16: {16}, 18: {18}}
CFG back edges: []
CFG loops:
{}
CFG node-to-loops:
{0: [], 16: [], 18: []}
CFG backbone:
{0}
-----------------------------IR DUMP: LightFunction-----------------------------
label 0:
    x = arg(0, name=x)                       ['x']
    y = arg(1, name=y)                       ['y']
    $const6.1.1 = const(int, 0)              ['$const6.1.1']
    $8compare_op.2 = y == $const6.1.1        ['$8compare_op.2', '$const6.1.1', 'y']
    bool12 = global(bool: <class 'bool'>)    ['bool12']
    $12pred = call bool12($8compare_op.2, func=bool12, args=(Var($8compare_op.2, 1475666804.py:3),), kws=(), vararg=None, varkwarg=None, target=None) ['$12pred', '$8compare_op.2', 'bool12']
    branch $12pred, 16, 18                   ['$12pred']
label 16:
    $const16.0 = const(int, 0)               ['$const16.0']
    $16return_const.1 = cast(value=$const16.

0.0

In [142]:
@jit
def JitHeavyFunction(x, y):
    ret = []
    for elemx in x:
        for elemy in y:
            ret.append(elemx**elemy + elemx*elemy)
            

def HeavyFunction(x, y):
    ret = []
    for elemx in x:
        for elemy in y:
            ret.append(elemx**elemy + elemx*elemy)
            
            
a = np.arange(1000)
b = np.linspace(0, 10, 100)

print(dis.dis(JitHeavyFunction))

  1           RESUME                   0

  3           BUILD_LIST               0
              STORE_FAST               2 (ret)

  4           LOAD_FAST                0 (x)
              GET_ITER
      L1:     FOR_ITER                36 (to L4)
              STORE_FAST               3 (elemx)

  5           LOAD_FAST                1 (y)
              GET_ITER
      L2:     FOR_ITER                27 (to L3)
              STORE_FAST               4 (elemy)

  6           LOAD_FAST                2 (ret)
              LOAD_ATTR                1 (append + NULL|self)
              LOAD_FAST_LOAD_FAST     52 (elemx, elemy)
              BINARY_OP                8 (**)
              LOAD_FAST_LOAD_FAST     52 (elemx, elemy)
              BINARY_OP                5 (*)
              BINARY_OP                0 (+)
              CALL                     1
              POP_TOP
              JUMP_BACKWARD           29 (to L2)

  5   L3:     END_FOR
              POP_TOP
              JUMP_BA

In [143]:
JitHeavyFunction(a, b)

os.environ['NUMBA_DUMP_CFG'] = '0'
os.environ['NUMBA_DUMP_IR'] = '0'

CFG adjacency lists:
{0: [12], 12: [16, 88], 16: [22], 22: [26, 80], 26: [22], 80: [12], 88: []}
CFG dominators:
{0: {0},
 12: {0, 12},
 16: {16, 0, 12},
 22: {16, 0, 12, 22},
 26: {16, 0, 22, 26, 12},
 80: {80, 16, 0, 22, 12},
 88: {88, 0, 12}}
CFG post-dominators:
{0: {0, 88, 12},
 12: {88, 12},
 16: {16, 80, 22, 88, 12},
 22: {80, 88, 12, 22},
 26: {80, 22, 88, 26, 12},
 80: {80, 88, 12},
 88: {88}}
CFG back edges: [(26, 22), (80, 12)]
CFG loops:
{12: Loop(entries={0}, exits={88}, header=12, body={12, 80, 16, 22, 26}),
 22: Loop(entries={16}, exits={80}, header=22, body={26, 22})}
CFG node-to-loops:
{0: [], 12: [12], 16: [12], 22: [22, 12], 26: [22, 12], 80: [12], 88: []}
CFG backbone:
{0, 88, 12}
---------------------------IR DUMP: JitHeavyFunction----------------------------
label 0:
    x = arg(0, name=x)                       ['x']
    y = arg(1, name=y)                       ['y']
    ret = build_list(items=[])               ['ret']
    $10get_iter.2 = getiter(value=x)         